# Accessing, Analyzing, & Visualizing TEMPO data through ArcGIS Image Services Programmatically (Geometry Envelope)

## Overview

This notebook demonstrates how to access and analyze data from the TEMPO NO₂ Hourly Tropospheric Vertical Column product, which has been processed into free, publicly available ArcGIS Image Services. These services are pre-filtered and analysis-ready, enabling users to extract and visualize NO₂ values over time without the need for specialized GIS software.

This notebook walks through the following steps:

- Select and define a TEMPO image service to query  
- Choose a time range and geographic area of interest  
- Retrieve and visualize NO₂ values within a defined polygon (bounding box)  
- Plot average NO₂ levels over time for the selected region  
- Explore the same imagery interactively using an ipyleaflet-based time slider map  

## Why ArcGIS Image Services?

Each TEMPO Image Service is hosted at a REST endpoint that offers multiple built-in capabilities via the [ArcGIS image service REST API](https://developers.arcgis.com/rest/services-reference/enterprise/image-service/). These endpoints allow users to:

- Programmatically query data values (e.g., with `getSamples`)  
- Retrieve rendered imagery tiles  
- Filter by dimensions like time and variables  
- Embed and visualize the data in custom web or Python-based applications  

These services make it easier for non-GIS users to work with complex satellite data products using standard tools like Python.

## Prerequisites

No Esri software or GIS tools are required. All steps in this notebook use **open-source Python libraries** to access and analyze the data.

### Required Skills:
- Basic Python (functions, loops, plotting)
- Familiarity with air quality or remote sensing concepts (e.g., NO₂ columns, temporal analysis)

### Required Python Libraries:
- [requests](https://github.com/psf/requests) - for sending HTTP requests to service API
- [matplotlib](http://matplotlib.org/) - for creating plots and visualizations
- [json](https://docs.python.org/3/library/json.html) - convert dictionaries into JSON strings
- [datetime](https://docs.python.org/3/library/datetime.html) - Handles date and time objects
- [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet/blob/master/python/ipyleaflet/README.md) - for visualization in interactive mapper
- [ipywidgets](https://pypi.org/project/ipywidgets/) – for map controls and user interaction  

### Data & Scope
Each TEMPO ArcGIS image service has a portal page with detailed descriptions on the service, the filtering applied, geographic and temporal coverage, as well as access to the online map viewer to view the image service. It is strongly recommended to read over the service description to ensure understanding of the data.

The TEMPO image services are available in the Esri Living Atlas of the World:
* [NO2](https://www.arcgis.com/home/item.html?id=6a1bdd0c076d499da69e867732ed2ab7)
* [HCHO](https://www.arcgis.com/home/item.html?id=27947c9d5d5f417b8b46a9d75a084549)
* [Ozone Total](https://www.arcgis.com/home/item.html?id=b6a2f0ebfbbc424aa58ef13af0a3bd6c)

The example in the notebook uses:

**Product**: TEMPO_NO2_L3_V03 (Hourly NO₂ Tropospheric Column)  
**Resolution**: ~2.1 × 4.4 km, ourly during daylight  
**Coverage**: North America   
**Example Region**: Hampton Roads Virginia  
**Time Range**: May 20–25, 2024  
  
*Methods apply to other TEMPO products (formaldehyde, ozone, etc.) and regions within North America.*

### Notebook Author / Affiliation
- Author: Atmospheric Science Data Center
- Questions? Please post questions on the [NASA Earthdata Forum](https://forum.earthdata.nasa.gov/)

# Create Time Graph - ArcGIS Image Service: Get Samples

## 1. Setup
Install Python packages, as necessary (NOTE: Google Colab appears to have all the packages pre-installed)
Install Python packages if not available; you may want to restart the kernel after installation to ensure changes are implemented

In [1]:
#!pip install requests pandas matplotlib ipyleaflet ipywidgets

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import json
from datetime import datetime

## 2. Convert Human-Readable Dates to Milliseconds Since Epoch

Next we will run the `convert_to_milliseconds()` function. This function takes a date-time string in the format `'YYYY-MM-DD HH:MM:SS'` and converts it to Unix time in **milliseconds**. This format is required by the NASA ArcGIS ImageServer API, which expects time inputs in millisecond timestamps for querying NO₂ data over a given time range.


In [ ]:
def convert_to_milliseconds(date_time_str):
    """Converts a date-time string in 'YYYY-MM-DD HH:MM:SS' format to milliseconds since epoch."""
    dt = datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S')
    milliseconds_since_epoch = int(dt.timestamp() * 1000)
    return milliseconds_since_epoch

## 3. Define Input Parameters
This cell sets up the key user inputs for the analysis:
- The URL of the NASA ImageServer layer that provides hourly NO₂ tropospheric column data.
- The name of the variable we want to query (`NO2_Troposphere`).
- The time range of interest.

The start and end times are then converted to milliseconds with our previously defined function.

In [ ]:
# User inputs
image_service_url = "https://gis.earthdata.nasa.gov/image/rest/services/C2930763263-LARC_CLOUD/TEMPO_NO2_L3_V03_HOURLY_TROPOSPHERIC_VERTICAL_COLUMN/ImageServer"
variable_name = "NO2_Troposphere"  # Example: "T2M" - TODO retrieve list of variables from MultidimensionalInfo endpoint
start_date_time_str = "2024-05-20 12:00:00" #in 'YYYY-MM-DD HH:MM:SS' format
end_date_time_str = "2024-05-25 12:00:00" #in 'YYYY-MM-DD HH:MM:SS' format

# Convert user input dates to milliseconds since epoch
start_time = convert_to_milliseconds(start_date_time_str)
end_time = convert_to_milliseconds(end_date_time_str)

Next, we will check to ensure our function conversion was successful

In [ ]:
print(start_time)
print(end_time)

### 3.1 Define Spatial and API Request Parameters
Next, we will construct our request to the NASA ImageServer API.

- A geographic bounding box (`geometry_envelope`) is defined using latitude and longitude limits, with WGS84 coordinates (`wkid: 4326`).
- API parameters are assembled into the `params` dictionary, including the geometry, time range, variable name, interpolation method, and response format.
- The bounding box will return all sample points within the specified area for the given time range.

The `geometry` is serialized using `json.dumps()` because the API requires JSON-formatted strings for complex geometry inputs.

In [ ]:
base_url = image_service_url+"/getSamples/"

geometry_envelope = {
    "xmin": -76.6,
    "ymin": 37.1,
    "xmax": -76.3,
    "ymax": 37.3,
    "spatialReference": { "wkid": 4326 }
}

params = {
    "geometry": json.dumps(geometry_envelope),
    "geometryType": "esriGeometryEnvelope",
    "sampleDistance": "",
    "sampleCount": "",
    "mosaicRule": f'{{"multidimensionalDefinition":[{{"variableName":"{variable_name}"}}]}}',
    "pixelSize": "",
    "returnFirstValueOnly": "false",
    "interpolation": "RSP_NearestNeighbor",
    "outFields": "",
    "sliceId": "",
    "time": f"{start_time},{end_time}",
    "f": "pjson"
}

## 4. Retreive data values for our area of interest for selected time period
https://gis.earthdata.nasa.gov/image/rest/services/C2930763263-LARC_CLOUD/TEMPO_NO2_L3_V03_HOURLY_TROPOSPHERIC_VERTICAL_COLUMN/ImageServer


### 4.1 Send API Request and Preview Sample Data

This cell sends a GET request to the NASA ArcGIS ImageServer using the `getSamples` endpoint. The full response is stored in the `data` variable. To keep the output manageable, only the first 3 sample entries (if any) are printed. This helps verify the API call succeeded and returned usable data.

In [ ]:
response = requests.get(base_url, params=params)
data = response.json()

# Use print(data) to view all data (it will be long)
# print(data)

# Use the following to print the first 3 samples of data
samples = data.get("samples", [])
if samples:
    for i, sample in enumerate(samples[:3]):
        print(f"Sample {i+1}: {sample}")
else:
    print("No samples returned.")

### 4.2 Extract data into a dataframe
This cell extracts the relevant information from the API response and organizes it into a `pandas` DataFrame.

- It loops through each sample and attempts to extract the NO₂ value from the `"attributes"` field.
- Only **positive values** are kept, since negative values usually represent invalid or low-quality data (e.g., due to clouds or sensor issues).
- Invalid or missing values are safely skipped using a `try`–`except` block.
- Timestamps (`StdTime`) are converted from milliseconds to human-readable datetime format using `pandas.to_datetime()`.

In [ ]:
samples = []
for sample in data.get("samples", []):
    attributes = sample.get("attributes", {})
    no2_str = attributes.get(variable_name)

    try:
        no2_value = float(no2_str)
        if no2_value > 0:  
            samples.append({
                "StdTime": attributes["StdTime"],
                variable_name: no2_value
            })
    except (TypeError, ValueError):
        # Skip if value can't be converted to float
        continue

# Convert the list to a DataFrame
df = pd.DataFrame(samples)


# Convert StdTime from Unix timestamp (milliseconds) to datetime
df['StdTime'] = pd.to_datetime(df['StdTime'], unit='ms')

In [ ]:
df

## 5. Plot Average NO₂ Values Over Time

This cell calculates and visualizes the average NO₂ concentration across all valid sample points in the envelope for each timestamp.

- The data is grouped by time (`StdTime`), and the mean value is computed for each group.
- A line plot is used to display how average NO₂ levels change over time.
- This plot helps identify overall trends and patterns, such as increases or decreases in NO₂ over the selected period.

In [ ]:
# Finding the average values of the polygon area and plotting over time
df_avg = df.groupby('StdTime')[variable_name].mean().reset_index()

plt.figure(figsize=(10, 6))
plt.plot(df_avg['StdTime'], df_avg[variable_name], marker='o', linestyle='-')
plt.title(f'Average {variable_name} Over Time')
plt.xlabel('Time')
plt.ylabel(f'Average {variable_name}')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Create Map - ArcGIS Image Servce: Export Image

### 1. Import Required Libraries

In [ ]:
from ipyleaflet import Map, ImageService, basemaps,  WidgetControl
from ipywidgets import SelectionSlider, Layout, Label, VBox
from datetime import datetime, timezone
from ipywidgets import Output, HTML

### 2. Define Utility Functions for Time and Map Interaction

This cell defines two funcitons we will use in our map:

1. `convert_from_milliseconds()`  
   Converts a Unix timestamp in milliseconds into a readable ISO 8601 date-time string (e.g., `'2024-05-20T12:00:00Z'`). This format is useful for labeling or interpreting time values associated with imagery layers.

2. `on_click()`  
   Adds interactivity to the map. When a user:
   - **Clicks**, the geographic coordinates are printed to the console.
   - **Moves the mouse**, the live coordinates are displayed on the map in a floating label.

In [ ]:
def convert_from_milliseconds(milliseconds_since_epoch):
    """Converts milliseconds since epoch to a date-time string in 'YYYY-MM-DDTHH:MM:SSZ' format."""
    dt = datetime.fromtimestamp((milliseconds_since_epoch)/ 1000, tz=timezone.utc)
    date_time_str = dt.strftime('%Y-%m-%dT%H:%M:%SZ')
    return date_time_str


def on_click(**kwargs):
    if kwargs.get('type') == 'click':
        print(str(kwargs.get('coordinates')))
    
    if kwargs.get('type') == 'mousemove':
        latlng = kwargs.get('coordinates')
        lat, lng = latlng
        coordinates_label.value = f"Coordinates: ({lat:.5f}, {lng:.5f})"

### 3. Define Observation Times and Image Service URL

This cell provides a list of available observation times from the ImageServer, stored as Unix timestamps in milliseconds. These time values are used to populate the time slider on the interactive map, allowing users to view NO₂ imagery for specific hourly intervals. The ImageServer URL is also defined here, referencing the TEMPO hourly NO₂ product that will be displayed on the map.


In [ ]:
# The actual start times of observations from the datafiles
time_values = [
    1715683263000,
    1715685668000,
    1715688073000,
    1715690478000,
    1715692883000,
    1715695288000,
    1715698888000,
    1715702488000,
    1715706088000,
    1715709688000,
    1715713288000,
    1715716888000,
    1715720488000,
    1715724088000,
    1715726493000,
    1715728898000,
    1715731303000,
    1715733708000,
]

image_service_url = "https://gis.earthdata.nasa.gov/image/rest/services/C2930763263-LARC_CLOUD/TEMPO_NO2_L3_V03_HOURLY_TROPOSPHERIC_VERTICAL_COLUMN/ImageServer"


### 4. Create Interactive Map with ImageService and Time Slider

This final cell creates an interactive `ipyleaflet` map that displays hourly NO₂ imagery from the NASA ImageServer. The time slider allows users to select a specific observation timestamp, which dynamically updates the image layer. A label displays live mouse coordinates on hover, and a click listener prints coordinates to the console.

In [ ]:
# Initialize the map
m = Map(center=(47,-122), zoom=3, basemap=basemaps.Esri.WorldTopoMap)

tempo_image_service = ImageService(url=image_service_url, 
                                   rendering_rule={"rasterFunction":"torch_RGB"}, 
                                   format="jpgpng",
                                   opacity=0.5)


#creating a list with the UTC times with time_values for easy visualization of time
time_strings = [convert_from_milliseconds(t) for t in time_values]  

#creating a list of tuples to input in SelectionSlider's options for easy visualization of time
time_options = [(time_strings[i], time_values[i]) for i in range(len(time_values))]

#creating the slider
slider = SelectionSlider(description='Time:', options=time_options, layout=Layout(width='700px', height='20px'))

#creating a Label for the VBox
time_label = Label(value='Time Slider')

#A handler that will update the map everytime the user moves the slider
def update_image(change):

    tempo_image_service.time = [change.new,1715733708000]
    
#Listens to the slider's user input and helps update the map
slider.observe(update_image, 'value')

#creates a VBox to contain the slider and be placed in the map
vbox = VBox([slider, time_label])

#slider placed in bottomleft of the map
control = WidgetControl(widget=vbox, position='bottomleft')

# Output widget to listen to the user's mouse hovering over the map
output = Output()
controloutput = WidgetControl(widget=output, position='topright')

# Label widget to display coordinates
coordinates_label = HTML(value="Coordinates: ")
coordinates_control = WidgetControl(widget=coordinates_label, position='bottomright')

#add all widgets to the map
m.add(tempo_image_service)
m.add(control)
m.add(controloutput)
m.add(coordinates_control)

# when user hovers over the map coordinates_label gets updated and prints the coordinates where clicked
m.on_interaction(on_click)
m